In [122]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [127]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

# Scratch notebook for handling long stop to stop segments

* goal is to provide additional map detail for rural/express routes

In [128]:
## KART speedmap as displayed on site
itp_id = 148
analysis_date = dt.date(2022, 6, 1)

In [129]:
pbar = tqdm()

0it [00:00, ?it/s]

In [130]:
kart_analysis = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
less than 1km of data
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less t

In [131]:
# gdf = kart_analysis.routelines >> filter(_.shape_id == '3')
# shared_utils.rt_utils.map_line(gdf)

In [132]:
kart_analysis.rt_trips >> filter(_.route_short_name == '12')

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
266,148,0,2022-06-01,2644978890892601525,949,12,0,11,2021-07-02,2099-01-01,12,Avenal,None,3,12:27:19,Westbound,31.666138,Kings Area Rural Transit
267,148,0,2022-06-01,3494167986599370378,947,12,0,3,2021-07-02,2099-01-01,12,Avenal,None,3,07:26:32,Southbound,38.489930,Kings Area Rural Transit
268,148,0,2022-06-01,-4140891564404050759,950,12,0,27,2021-07-02,2099-01-01,12,Avenal,None,3,17:44:36.500000,Westbound,40.296843,Kings Area Rural Transit


In [141]:
# kart_analysis.stop_delay_view = kart_analysis.stop_delay_view >> mutate(last_loc = _.shape_meters.shift(1))

In [134]:
long_trip = kart_analysis.stop_delay_view >> filter(_.trip_id == '947') >> arrange(_.stop_sequence)

In [137]:
test_center = (long_trip >> arrange(_.shape_meters)).iloc[10:49, :]

In [153]:
# kart_analysis.stop_delay_view.arrival_time = kart_analysis.stop_delay_view.apply(lambda x: x.arrival_time if x.arrival_time else x.actual_time, axis = 1)

In [154]:
kart_analysis.stop_delay_view.arrival_time = kart_analysis.stop_delay_view.actual_time

In [155]:
kart_analysis.export_views_gcs()

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

In [156]:
import rt_filter_map_plot

In [157]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [158]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

found parquet


In [159]:
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [160]:
m = rt_day.segment_speed_map()

TopologyException: assigned depths do not match at 15912.256659238848 -191580.72014725895
TopologyException: assigned depths do not match at 32532.202493516623 -190255.07589254106
TopologyException: assigned depths do not match at 32532.202493516623 -190255.07589254106
/opt/conda/lib/python3.10/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:281: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [162]:
# m